Задание 5. Влияние ошибок округления на решение СЛАУ. Числа обусловленности.

In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import hilbert
import pandas as pd

In [2]:
def spectral_condition_number(matrix: np.ndarray) -> float:
    return LA.norm(matrix) * LA.norm(LA.inv(matrix))


def volume_condition_number(matrix: np.ndarray) -> float:
    volume = 1
    for row in matrix:
        volume *= LA.norm(row)
    return volume / abs(LA.det(matrix))


def angle_condition_number(matrix: np.ndarray) -> float:
    size = matrix.shape[0]
    inverse_matrix = LA.inv(matrix)
    return max(LA.norm(matrix[n]) * LA.norm(inverse_matrix[:, n]) for n in range(0, size))


condition_numbers = {
    'cond_s': spectral_condition_number,
    'cond_v': volume_condition_number,
    'cond_a': angle_condition_number
}

In [3]:
def condition_numbers_df(matrix: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame.from_dict({name: [func(matrix)] for name, func in condition_numbers.items()})


def variation_results_df(matrix: np.ndarray) -> pd.DataFrame:
    x = np.ones(matrix.shape[1])
    right_part = matrix @ x

    variation = [10 ** -p for p in range(2, 11)]
    data = []
    for v in variation:
        new_matrix, new_right_part = matrix + v, right_part + v
        new_x = LA.solve(new_matrix, new_right_part)
        error = LA.norm(x - new_x)
        data.append([v, error])
    return pd.DataFrame(data, columns=['смещение', 'погрешность'])

### Матрица Гильберта

In [4]:
matrix = hilbert(7)
display(condition_numbers_df(matrix))
display(variation_results_df(matrix))

,cond_s,cond_v,cond_a
0,4.817473e+08,1.421000e+22,7.162186e+07


,смещение,погрешность
0,1.000000e-02,2110.945204
1,1.000000e-03,299.838736
2,1.000000e-04,31.299715
3,1.000000e-05,3.143768
4,1.000000e-06,0.314515
5,1.000000e-07,0.031453
6,1.000000e-08,0.003145
7,1.000000e-09,0.000315
8,1.000000e-10,0.000031


### Диагональная матрица

In [5]:
matrix = np.identity(5)
matrix[0][0] = 1000

display(condition_numbers_df(matrix))
display(variation_results_df(matrix))

,cond_s,cond_v,cond_a
0,2000.00425,1.0,1.0


,смещение,погрешность
0,1.000000e-02,7.692235e-02
1,1.000000e-03,7.968121e-03
2,1.000000e-04,7.996801e-04
3,1.000000e-05,7.999681e-05
4,1.000000e-06,7.999969e-06
5,1.000000e-07,7.999998e-07
6,1.000000e-08,8.000001e-08
7,1.000000e-09,8.000001e-09
8,1.000000e-10,8.000002e-10


### Трехдиагональная матрица с диагональным преобладанием 

In [6]:
size = 10
matrix = 2 * np.identity(size) + -1 * np.eye(size, k=-1) + -1 * np.eye(size, k=1)

display(condition_numbers_df(matrix))
display(variation_results_df(matrix))

,cond_s,cond_v,cond_a
0,98.122373,589.090909,12.898203


,смещение,погрешность
0,1.000000e-02,1.569999e+00
1,1.000000e-03,2.970269e-01
2,1.000000e-04,3.261126e-02
3,1.000000e-05,3.293376e-03
4,1.000000e-06,3.296636e-04
5,1.000000e-07,3.296962e-05
6,1.000000e-08,3.296995e-06
7,1.000000e-09,3.296998e-07
8,1.000000e-10,3.296999e-08
